In [ ]:
#!pip install datasets
#!pip install transformers
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import datasets
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification
import torch
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
from tqdm import tqdm
import os
import evaluate
from sklearn.dummy import DummyClassifier
from datasets import load_dataset
from collections import Counter, defaultdict

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
type(imdb)

datasets.dataset_dict.DatasetDict

In [ ]:
imdb.keys()

dict_keys(['train', 'test', 'unsupervised'])

In [ ]:
type(imdb["train"])

datasets.arrow_dataset.Dataset

In [ ]:
imdb["train"]['text'][3], imdb["train"]['label'][3]

("This film was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The film has two strong elements and those are, (1) the realistic acting (2) the impressive, undeservedly good, photo. Apart from that, what strikes me most is the endless stream of silliness. Lena Nyman has to be most annoying actress in the world. She acts so stupid and with all the nudity in this film,...it's unattractive. Comparing to Godard's film, intellectuality has been replaced with stupidity. Without going too far on this subject, I would say that follows from the difference in ideals between the French and the Swedish society.<br /><br />A movie of its time, and place. 2/10.",
 0)

In [ ]:
split_ds = imdb['train'].train_test_split(test_size=0.2, stratify_by_column="label")

In [ ]:
split_ds['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})

In [ ]:
imdb['train'] = split_ds['train']
imdb['dev'] = split_ds['test']

In [ ]:
imdb["train"][1]

{'text': 'Is rich, ailing Elizabeth Taylor courting the Angel of Death on her island fortress in the Mediterranean, or is she just overreacting--or more precisely, overacting--as usual? Actually, both are applicable in director Joseph Losey\'s wandering, meandering mess called "Boom", appropriately titled since tempers in the lush, luxurious setting are nearly ready to explode. Richard Burton climbs Taylor\'s mountain uninvited; she dresses him in a samurai\'s robe complete with saber. Though great-looking in widescreen, the picture is otherwise quite deadly, a failure even Liz \'n Dick-philes should shun (the stars\' collective "what the hell!" attitude to their late-\'60s film careers reached an ego-mad nadir here). Pointless, confused, and maddening, "Boom" is a catastrophe--although screenwriter Tennessee Williams, who adapted his own unsuccessful play "The Milk Train Doesn\'t Stop Here Anymore", was said to be quite fond of it! * from ****',
 'label': 0}

In [ ]:
imdb.type()

AttributeError: ignored

In [ ]:
imdb["test"]

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

## Convert datasets to dataframes to use with sklearn

In [ ]:
train_df = defaultdict(list)
for item in tqdm(imdb['train']):
    train_df['text'].append(item['text'])
    train_df['label'].append(item['label'])    
train_df = pd.DataFrame(train_df)




  0%|          | 0/20000 [00:00<?, ?it/s]


  4%|▍         | 803/20000 [00:00<00:02, 8029.43it/s]


  8%|▊         | 1656/20000 [00:00<00:02, 8321.03it/s]


 13%|█▎        | 2515/20000 [00:00<00:02, 8442.89it/s]


 17%|█▋        | 3425/20000 [00:00<00:01, 8700.93it/s]


 21%|██▏       | 4296/20000 [00:00<00:01, 8262.73it/s]


 26%|██▌       | 5126/20000 [00:00<00:01, 8261.74it/s]


 30%|██▉       | 5966/20000 [00:00<00:01, 8304.33it/s]


 34%|███▍      | 6887/20000 [00:00<00:01, 8585.73it/s]


 39%|███▉      | 7786/20000 [00:00<00:01, 8710.02it/s]


 43%|████▎     | 8659/20000 [00:01<00:01, 8628.01it/s]


 48%|████▊     | 9523/20000 [00:01<00:01, 8621.14it/s]


 52%|█████▏    | 10399/20000 [00:01<00:01, 8660.70it/s]


 57%|█████▋    | 11308/20000 [00:01<00:00, 8788.22it/s]


 61%|██████    | 12188/20000 [00:01<00:00, 8768.47it/s]


 65%|██████▌   | 13071/20000 [00:01<00:00, 8784.26it/s]


 70%|██████▉   | 13950/20000 [00:01<00:00, 8372.57it/s]


 74%|███████▍  | 14792/20000 [00:01<0

In [ ]:
test_df = defaultdict(list)
for item in tqdm(imdb['test']):
    test_df['text'].append(item['text'])
    test_df['label'].append(item['label'])    
test_df = pd.DataFrame(test_df)    




  0%|          | 0/25000 [00:00<?, ?it/s]


  8%|▊         | 2059/25000 [00:00<00:01, 20582.12it/s]


 17%|█▋        | 4294/25000 [00:00<00:00, 21616.84it/s]


 26%|██▌       | 6456/25000 [00:00<00:00, 20482.27it/s]


 34%|███▍      | 8532/25000 [00:00<00:00, 20587.45it/s]


 43%|████▎     | 10657/25000 [00:00<00:00, 20820.37it/s]


 51%|█████     | 12784/25000 [00:00<00:00, 20970.23it/s]


 64%|██████▍   | 15973/25000 [00:00<00:00, 24504.15it/s]


 78%|███████▊  | 19487/25000 [00:00<00:00, 27867.10it/s]


100%|██████████| 25000/25000 [00:00<00:00, 25688.35it/s]


# Dummy classifiers

In [ ]:
mf_dummy_clf = DummyClassifier(strategy="most_frequent")
mf_dummy_clf.fit(train_df.text, train_df.label)
mf_dummy_test_preds = mf_dummy_clf.predict(test_df.text)
precision_recall_fscore_support(test_df.label, mf_dummy_test_preds, average='binary')

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0, 0.0, 0.0, None)

In [ ]:
random_dummy_clf = DummyClassifier(strategy="stratified")
random_dummy_clf.fit(train_df.text, train_df.label)
random_dummy_test_preds = random_dummy_clf.predict(test_df.text)
precision_recall_fscore_support(test_df.label, random_dummy_test_preds, average='binary')

(0.4986413043478261, 0.49912, 0.49888053734207577, None)

# Standard sklearn classifier(s)

In [ ]:
# Convert text to features using a simple tf-idf
# min_df = is the minimum occurrences for a word to a feature
vectorizer = TfidfVectorizer(min_df=10)
X_train = vectorizer.fit_transform(train_df.text)

# Train a Logisitic Regression classifier
clf = LogisticRegression()
clf.fit(X_train, train_df.label)

# Test the classifier by converting test data to features
X_test = vectorizer.transform(test_df.text)
y_pred = clf.predict(X_test)        

# We'll using binary F1 to test whether we can detect sarcastic comments
y_test = test_df.label

In [ ]:
precision_recall_fscore_support(test_df.label, y_pred, average='binary')

(0.8814832574122912, 0.8824, 0.881941390476952, None)

# Fancy Huggingface classifiers

We'll use [MiniLM](https://huggingface.co/microsoft/MiniLM-L12-H384-uncased) which is a compressed version of a BERT-like model that is faster to train

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Verify there's at least one GPU sitting around (or not if you want to try CPU!)
torch.cuda.device_count()

1

### Prepare the text

We need to tokenize the data. Each large language model (LLM) has its own tokenizer, so we will have to specify how the model expects the text to be turned into tokens. In huggingface, we'll specify a LLM using a string name, e.g., "microsoft/MiniLM-L12-H384-uncased" for MiniLM. You can see more details on different possible model on Huggingface's [model repo](https://huggingface.co/models)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128)
tokenized_imdb = imdb.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
type(tokenized_imdb["train"]) #, tokenized_imdb["train"].head

datasets.arrow_dataset.Dataset

### Train the model

In [ ]:
# Huggingface will generate model checkpoints and these can 
# take up a lot of space, so be sure to set this directory somewhere where
# you can check

output_dir = '.'

# This code block is used so we can have a deterministic training procedure
# (i.e., get the same result each time)
model_name = "microsoft/MiniLM-L12-H384-uncased"
def model_init():
      return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
        
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    do_eval=True,
    seed=1234,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    logging_dir=output_dir + 'logs/',
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

trainer = Trainer(
    model_init=model_init,    
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["dev"],    
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
tokenized_ds

In [ ]:
dev_logits = trainer.predict(tokenized_ds["dev"])
print(dev_logits.predictions.shape, dev_logits.label_ids.shape)
dev_preds = np.argmax(dev_logits.predictions, axis=-1)
precision_recall_fscore_support(dev_df.label, dev_preds, average='binary')

In [ ]:
test_logits = trainer.predict(tokenized_ds["test"])
print(test_logits.predictions.shape, test_logits.label_ids.shape)
test_preds = np.argmax(test_logits.predictions, axis=-1)
precision_recall_fscore_support(test_df.label, test_preds, average='binary')